## 💼 "All-Weather" Risk-Parity Portfolio Framework

### 📥 Setups and ETFs Selection

Installation and Import of required packages

In [197]:
# !pip install -r requirements.txt

In [198]:
import yfinance as yf
import pandas as pd
import numpy as np
import plotly.graph_objects as go

Initialization of portfolio parameters

In [199]:
TICKERS: list[str] = ['SPY', 'TLT', 'GLD', 'TIP', 'BIL']  # Equities, Fixed Income, Commodities, Inflation-protected, Cash
START_DATE: str = '2015-01-01'
END_DATE: str = '2025-06-27'
VOL_LOOKBACK: int = 126  # Volatility Lookback Period Length

### 🏗️ Data Download

Function for fetching daily price data

In [200]:
def get_etf_data(tickers: list[str], start: str, end: str):
    """
    Fetches financial data from Yahoo Finance API for given tickers and date range.
    
    Parameters:
        tickers (list[str]): U.S. equity ticker symbols
        start_date (str): Start date in 'YYYY-MM-DD' format
        end_date (str): End date in 'YYYY-MM-DD' format

    Returns:
        pd.DataFrame: DataFrame with date-indexed adjusted closing price data
    """
    
    data = yf.download(tickers, start, end, auto_adjust=True)['Close']
    return data.dropna()

ETF Prices

In [201]:
prices = get_etf_data(TICKERS, START_DATE, END_DATE)

[*********************100%***********************]  5 of 5 completed


In [202]:
prices.head()

Ticker,BIL,GLD,SPY,TIP,TLT
Date,,,,,
2015-01-02,76.333504,114.080002,171.568069,85.979080,96.881805
2015-01-05,76.350182,115.800003,168.469559,86.047745,98.403671
2015-01-06,76.333504,117.120003,166.882767,86.055389,100.176643
2015-01-07,76.333504,116.430000,168.962372,86.124008,99.978798
2015-01-08,76.333504,115.940002,171.960571,85.986732,98.654785


### 🧮 Volatility and Weight Calculation

Functions for calculating rolling volatility and daily risk-parity weights

In [203]:
def calculate_rolling_volatility(tickers: list[str], start: str, end: str, window: int):
    """
    Calculates rolling 126-day volatility for given tickers and date range.
    
    Parameters:
        tickers (list[str]): U.S. equity ticker symbols
        start_date (str): Start date in 'YYYY-MM-DD' format
        end_date (str): End date in 'YYYY-MM-DD' format
        window (int): Length of lookback period in number of days

    Returns:
        pd.DataFrame: DataFrame with date-indexed rolling 126-day volatility
    """

    # Fetch price data
    data = get_etf_data(tickers, start, end)

    # Compute daily return
    returns = data.pct_change()

    # Compute rolling volatility
    rolling_vol = returns.rolling(window=window).std()

    return rolling_vol

In [204]:
def calculate_daily_risk_parity_weights(tickers: list[str] = TICKERS, start: str = START_DATE, end: str = END_DATE, window: int = VOL_LOOKBACK):
    """
    Calculates daily risk-parity weights for given tickers and date range.
    
    Parameters:
        tickers (list[str]): U.S. equity ticker symbols
        start (str): Start date in 'YYYY-MM-DD' format
        end (str): End date in 'YYYY-MM-DD' format
        window (int): Length of lookback period in number of days

    Returns:
        pd.DataFrame: DataFrame with date-indexed risk-parity weights
    """

    # Fetch rolling volatility
    vol = calculate_rolling_volatility(tickers, start, end, window)

    # Invert volatility
    inv_vol = 1 / vol.replace(0, np.nan)  # Replace zero vol with NaN to avoid inf

    # Normalize rows so weights sum to 1
    weights = inv_vol.div(inv_vol.sum(axis=1), axis=0)  # Fill NaN weights with zero

    return weights

Portfolio Weights

In [205]:
portfolio_weights = calculate_daily_risk_parity_weights()

[*********************100%***********************]  5 of 5 completed


In [206]:
portfolio_weights.dropna().head()

Ticker,BIL,GLD,SPY,TIP,TLT
Date,,,,,
2015-07-06,0.907861,0.017557,0.020897,0.039067,0.014618
2015-07-07,0.907434,0.017628,0.021322,0.038931,0.014685
2015-07-08,0.907992,0.017646,0.020912,0.038711,0.014738
2015-07-09,0.908283,0.017672,0.021138,0.038347,0.014560
2015-07-10,0.908247,0.017683,0.021375,0.038171,0.014524


### 📜 Backtesting

Functions for calculating risk-parity portfolio returns and performance metrics

In [207]:
def calculate_portfolio_returns(weights: pd.DataFrame, closing_prices: pd.DataFrame):
    """
    Calculates daily risk-parity portfolio returns for given weights and total-return ETF prices
    
    Parameters:
        weights (pd.DataFrame): Daily risk-parity weights of the portfolio
        closing_prices (pd.DataFrame): Daily adjusted closing price of each ETF

    Returns:
        pd.DataFrame: DataFrame with date-indexed daily returns
    """

    # Compute daily return
    returns = closing_prices.pct_change().dropna()

    # Shift weights to avoid look-ahead bias
    weights_lagged = weights.shift(1).dropna()

    # Align returns with weights
    returns = returns.loc[weights_lagged.index]

    # Calculate portfolio returns
    portfolio_returns = (weights_lagged * returns).sum(axis=1).to_frame()

    # Rename the column to 'returns'
    portfolio_returns.columns = ['returns']

    # Calculate cumulative returns
    portfolio_returns['cumulative_returns'] = (1 + portfolio_returns['returns']).cumprod()

    # Suffix asset return columns with '_return'
    returns.columns = [col + '_return' for col in returns.columns]

    return pd.concat([returns, portfolio_returns], axis=1)

In [208]:
def calculate_portfolio_metrics(portfolio_returns: pd.DataFrame, risk_free_rate: float = 0.0) -> dict:
    """
    Computes annualized return, volatility, Sharpe ratio, and max drawdown, assuming 252 trading days/year

    Parameters:
        portfolio_returns (pd.DataFrame): Daily returns of portfolio
        risk_free_rate (float): Daily risk-free rate in decimal (e.g., 0.0 or 0.0001 for 2.5% annualized)

    Returns:
        dict: Dictionary of metrics
    """

    # Calculates number of years of data
    num_years = (portfolio_returns.index[-1] - portfolio_returns.index[0]).days / 365.25

    annualized_return = float(portfolio_returns['cumulative_returns'].iloc[-1] ** (1 / num_years) - 1)

    annualized_volatility = float(portfolio_returns['returns'].std() * np.sqrt(252))

    sharpe_ratio = float(((portfolio_returns['returns'] - risk_free_rate).mean() / portfolio_returns['returns'].std()) * np.sqrt(252))

    max_drawdown = float((portfolio_returns['cumulative_returns'] / portfolio_returns['cumulative_returns'].cummax() - 1).min())

    return {
        'Annualized Return': annualized_return,
        'Annualized Volatility': annualized_volatility,
        'Sharpe Ratio': sharpe_ratio,
        'Max Drawdown': max_drawdown
    }

Function for backtesting

In [209]:
def backtest_portfolio(weights: pd.DataFrame, closing_prices: pd.DataFrame):
    """
    Backtest risk-parity portfolio by calculating portfolio metrics
    
    Parameters:
        weights (pd.DataFrame): Daily risk-parity weights of the portfolio
        closing_prices (pd.DataFrame): Daily adjusted closing price of each ETF

    Returns:
        Tuple[pd.DataFrame, dict]: Daily portfolio returns as DataFrame and summary performance metrics as a dictionary
    """

    returns = calculate_portfolio_returns(weights, closing_prices)

    metrics = calculate_portfolio_metrics(returns)

    return returns, metrics

Functions for calculating full-history and rolling VaR/CVaR

In [210]:
def calculate_full_hist_var_cvar(returns: pd.DataFrame, confidence: float = 0.95):
    """
    Calculate full-history Value-at-Risk and CVaR for given portfolio returns and confidence level
    
    Parameters:
        returns (pd.DataFrame): Daily returns of portfolio
        confidence (float): Confidence level of calculation (default to 0.95)

    Returns:
        dict[str, float]: Full-history VaR and CVaR
    """

    var = np.percentile(returns['returns'], 100 * (1 - confidence))
    cvar = returns['returns'][returns['returns'] <= var].mean()
    
    return {'VaR': float(var), 'CVaR': float(cvar)}

In [211]:
def calculate_rolling_var_cvar(returns: pd.DataFrame, period: int = 252, confidence: float = 0.95):
    """
    Calculate rolling Value-at-Risk and CVaR for given portfolio returns and confidence level
    
    Parameters:
        returns (pd.DataFrame): Daily returns of portfolio
        period (int): Length of lookback period in number of days (default to 252)
        confidence (float): Confidence level of calculation (default to 0.95)

    Returns:
        pd.DataFrame: Date-Indexed dataframe of rolling VaR and CVaR
    """

    rolling_var = []
    rolling_cvar = []

    for i in range(period, len(returns)):
        window_returns = returns[i - period:i]

        var_and_cvar = calculate_full_hist_var_cvar(window_returns)

        rolling_var.append(var_and_cvar['VaR'])
        rolling_cvar.append(var_and_cvar['CVaR'])
    
    dates = returns.index[period:]
    rolling_var_series = pd.Series(rolling_var, index = dates, name = 'VaR')
    rolling_cvar_series = pd.Series(rolling_cvar, index = dates, name = 'CVaR')

    return pd.concat([rolling_var_series, rolling_cvar_series], axis=1)

Returns and Performance Metrics

In [212]:
portfolio_returns, portfolio_metrics = backtest_portfolio(portfolio_weights, prices)

var_and_cvar = calculate_full_hist_var_cvar(portfolio_returns)

rolling_var_and_cvar = calculate_rolling_var_cvar(portfolio_returns)

In [213]:
portfolio_returns.head()

,BIL_return,GLD_return,SPY_return,TIP_return,TLT_return,returns,cumulative_returns
Date,,,,,,,
2015-07-07,-0.000219,-0.011601,0.006289,0.003287,0.009395,-0.000005,0.999995
2015-07-08,0.000000,0.002979,-0.016777,0.001240,0.008637,-0.000130,0.999864
2015-07-09,0.000000,0.002431,0.001809,-0.006456,-0.019703,-0.000460,0.999405
2015-07-10,0.000000,0.001167,0.012591,-0.004717,-0.015858,-0.000125,0.999280
2015-07-13,0.000000,-0.004485,0.011038,-0.003488,-0.003102,-0.000022,0.999258


In [214]:
portfolio_metrics

{'Annualized Return': 0.02175068243304934,
 'Annualized Volatility': 0.006159089144709818,
 'Sharpe Ratio': 3.5020761055845666,
 'Max Drawdown': -0.007586120205834712}

In [215]:
var_and_cvar

{'VaR': -0.0005420441358419241, 'CVaR': -0.0007485521371345981}

In [216]:
rolling_var_and_cvar.head()

,VaR,CVaR
Date,,
2016-07-06,-0.000549,-0.000717
2016-07-07,-0.000549,-0.000717
2016-07-08,-0.000549,-0.000717
2016-07-11,-0.000549,-0.000717
2016-07-12,-0.000549,-0.000717


### 📈 Performance Plots

Functions for plotting equity curve, weights evolution, and rolling VaR and CVaR

In [217]:
def plot_equity_curve(returns: pd.DataFrame):
    """
    Plot equity curve for given portfolio returns
    
    Parameters:
        portfolio_returns (pd.DataFrame): Daily returns of portfolio
    """
    
    cumulative_returns = (1 + returns['returns']).cumprod()

    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=cumulative_returns.index,
        y=cumulative_returns,
        mode='lines',
        name='Equity Curve',
        line=dict(color='green')
    ))

    fig.update_layout(
        title='Portfolio Equity Curve',
        xaxis_title='Date',
        yaxis_title='Cumulative Return (per $1)',
        template='plotly_white'
    )
    fig.show()

In [230]:
def plot_weights(weights: pd.DataFrame):
    """
    Plot weight evolution of each ETF over time
    
    Parameters:
        weights (pd.DataFrame): Daily risk-parity weights of the portfolio
    """
    
    fig = go.Figure()

    for col in weights.columns:
        fig.add_trace(go.Bar(
            x=weights.index,
            y=weights[col],
            name=col,
            hovertemplate=f'%{{x}}<br>{col}: %{{y:.2f}}<extra></extra>',
        ))

    fig.update_layout(
        barmode='stack',
        title=dict(
            text="Portfolio Weights over Time",
            font=dict(size=18, family="Arial Black"),
            x=0.5
        ),
        xaxis_title='Date',
        yaxis_title='Weights',
        yaxis=dict(range=[0, 1]),
        template='simple_white',
        legend=dict(
            orientation='v',
            yanchor='top',
            y=0.99,
            xanchor='left',
            x=0.01
        ),
        bargap=0,
    )

    fig.show()

In [219]:
def plot_rolling_var_and_cvar(rolling_var_and_cvar_series: pd.DataFrame):
    """
    Plot rolling VaR and CVaR

    Parameters:
        rolling_var_and_cvar_series (pd.DataFrame): Date-indexed rolling VaR and CVaR
    """
    
    # Create plot
    fig = go.Figure()

    # Add curves
    fig.add_trace(go.Scatter(x=rolling_var_and_cvar_series.index, y=rolling_var_and_cvar_series['VaR'], name='Rolling VaR (95%)'))
    fig.add_trace(go.Scatter(x=rolling_var_and_cvar_series.index, y=rolling_var_and_cvar_series['CVaR'], name='Rolling CVaR (95%)'))

    # Layout
    fig.update_layout(title="Rolling 252-Day VaR and CVaR", xaxis_title="Date", yaxis_title="Return", template="plotly_white")
    fig.show()

Plots

In [220]:
plot_equity_curve(portfolio_returns)

In [231]:
plot_weights(portfolio_weights)

In [ ]:
plot_rolling_var_and_cvar(rolling_var_and_cvar)

### 🧪 Stress-Testing Under Extreme Scenarios

Stress Scenarios Parameters

In [ ]:
STRESS_EVENTS: dict[str, tuple[str]] = {
    "March 2020 COVID-19 Crash (Feb-Apr 2020)": ("2020-02-19", "2020-03-23"),
    "2022 Russian-Ukraine War sell-off": ("2022-02-16", "2022-03-15"),
    "2025 Feb-April sell-off": ("2025-02-01", "2025-04-08")
}

EXTREME_SHOCK = {'SPY': -0.3, 'TLT': 0.1, 'GLD': -0.15}  # –30% equity, +10% rates move, –15% commodity move

Function for stress-testing the portfolio by recalculating returns and performance metrics

In [ ]:
def stress_test_period(weights: pd.DataFrame, closing_prices: pd.DataFrame, start_date: str, end_date: str):
    """
    Backtest risk-parity portfolio by calculating portfolio metrics
    
    Parameters:
        weights (pd.DataFrame): Daily risk-parity weights of the portfolio
        closing_prices (pd.DataFrame): Daily adjusted closing price of each ETF
        start_date (str): Start of stress period (YYYY-MM-DD).
        end_date (str): End of stress period (YYYY-MM-DD).

    Returns:
        Tuple[pd.DataFrame, dict[str, float|int]]: Daily portfolio returns under stress as DataFrame and summary performance metrics as a dictionary
    """

    # Calculate portfolio's entire daily returns
    full_returns = calculate_portfolio_returns(weights, closing_prices)

    # Get portfolio's returns and cumulative returns during stress window
    stress_returns = full_returns.loc[start_date:end_date].copy()

    # Calculate portfolio's performance metrics during stress window
    stress_metrics = calculate_portfolio_metrics(stress_returns)

    # Add cumulative return into metrics
    stress_metrics['Cumulative Return'] = float(stress_returns['cumulative_returns'].iloc[-1] / stress_returns['cumulative_returns'].iloc[0] - 1)
    
    # Find pre-stress peak
    pre_stress_peak = full_returns['cumulative_returns'].loc[:start_date].max()

    # Calculate portfolio's post_stress returns
    post_stress_returns = full_returns['cumulative_returns'].loc[end_date:]

    # Calculate recovery date and time
    recovery_date = post_stress_returns[post_stress_returns >= pre_stress_peak].first_valid_index()
    recovery_time = post_stress_returns.index.get_loc(recovery_date) if recovery_date else None

    # Add metric into dictionary
    stress_metrics['Recovery Time (Trading days)'] = recovery_time

    return stress_returns, stress_metrics

Function for simulating shock

In [ ]:
def simulate_shock(weights: pd.DataFrame, shock: dict[str, float], date: str | None = None):
    """
    Simulate extreme shock to estimate portfolio loss for given weights and day
    
    Parameters:
        weights (pd.DataFrame): Daily risk-parity weights of the portfolio
        shock (dict[str, float]): The shock to each asset
        date (str | None): Date of to simulate the shock for (default to first day)

    Returns:
        float: Portoflio loss from the shock
    """

    return float((weights.dropna().loc[date] * pd.Series(shock)).sum()) if date else float((weights.dropna().iloc[0] * pd.Series(shock)).sum())

Stress Returns and Performance Metrics

In [ ]:
stresses_performances = {}
for event in STRESS_EVENTS:
    stress_returns, stress_metrics = stress_test_period(portfolio_weights, prices, STRESS_EVENTS[event][0], STRESS_EVENTS[event][1])
    stresses_performances[event] = (stress_returns, stress_metrics)

shock_loss = simulate_shock(portfolio_weights, EXTREME_SHOCK)

In [ ]:
stresses_performances["March 2020 COVID-19 Crash (Feb-Apr 2020)"][0].head()

,BIL_return,GLD_return,SPY_return,TIP_return,TLT_return,returns,cumulative_returns
Date,,,,,,,
2020-02-19,0.000000,0.005831,0.004781,-0.000421,-0.000069,0.000140,1.075145
2020-02-20,0.000000,0.004085,-0.004108,0.001937,0.007834,0.000163,1.075320
2020-02-21,0.000219,0.015025,-0.010298,0.001849,0.009340,0.000444,1.075798
2020-02-24,-0.000109,0.008985,-0.033165,0.004614,0.014928,-0.000123,1.075666
2020-02-25,0.000000,-0.017874,-0.030302,-0.001670,0.005325,-0.000665,1.074951


In [ ]:
stresses_performances["March 2020 COVID-19 Crash (Feb-Apr 2020)"][1]

{'Annualized Return': 1.2107948366581969,
 'Annualized Volatility': 0.013037335926716704,
 'Sharpe Ratio': -0.5073684426591096,
 'Max Drawdown': -0.006674201287700687,
 'Cumulative Return': -0.000776951182513641,
 'Recovery Time (Trading days)': 1}

In [ ]:
stresses_performances["2022 Russian-Ukraine War sell-off"][0].head()

,BIL_return,GLD_return,SPY_return,TIP_return,TLT_return,returns,cumulative_returns
Date,,,,,,,
2022-02-16,0.000000,0.010284,0.001121,0.001948,0.005927,0.000262,1.080358
2022-02-17,-0.000109,0.013668,-0.021362,0.002268,0.007439,-0.000007,1.080350
2022-02-18,0.000109,-0.000733,-0.006475,0.003314,0.010527,0.000230,1.080599
2022-02-22,0.000000,0.002089,-0.010732,0.003061,0.002604,0.000032,1.080633
2022-02-23,-0.000109,0.004507,-0.017738,0.000723,-0.013782,-0.000341,1.080264


In [ ]:
stresses_performances["2022 Russian-Ukraine War sell-off"][1]

{'Annualized Return': 1.8607443627717881,
 'Annualized Volatility': 0.006072287887468664,
 'Sharpe Ratio': 1.4597004845710506,
 'Max Drawdown': -0.0017305694068977528,
 'Cumulative Return': 0.0004054724621627148,
 'Recovery Time (Trading days)': 167}

In [ ]:
stresses_performances["2025 Feb-April sell-off"][0].head()

,BIL_return,GLD_return,SPY_return,TIP_return,TLT_return,returns,cumulative_returns
Date,,,,,,,
2025-02-03,0.000284,0.005337,-0.006730,0.002315,0.008142,0.000484,1.217063
2025-02-04,0.000000,0.009848,0.006708,0.001663,0.003062,0.000391,1.217538
2025-02-05,0.000109,0.006210,0.004055,0.004426,0.016510,0.000764,1.218468
2025-02-06,0.000219,-0.002650,0.003476,-0.001744,-0.000445,0.000116,1.218609
2025-02-07,0.000219,0.001784,-0.009154,-0.002667,-0.006455,-0.000181,1.218388


In [ ]:
stresses_performances["2025 Feb-April sell-off"][1]

{'Annualized Return': 2.1747324974218722,
 'Annualized Volatility': 0.006710239668707343,
 'Sharpe Ratio': 5.281468951703269,
 'Max Drawdown': -0.002434189534879727,
 'Cumulative Return': 0.005998629687241408,
 'Recovery Time (Trading days)': 0}

In [ ]:
print("Shock Loss:", shock_loss)

Shock Loss: -0.007440963065621052


### 📉 Stress Performance Plots

Function for plotting equity curve overlays alongisde full-history curve for all stress windows

In [ ]:
def plot_equity_curve_with_stress(portfolio_returns: pd.DataFrame, stress_windows: dict[str, tuple[str]]):
    """
    Plot full equity curve with overlays for stress scenarios.

    Parameters:
        portfolio_returns (pd.DataFrame): Portfolio's entire daily returns and equity 
        stress_windows (dict[str, tuple[str]]): Dict of {label: (start_date, end_date)}
    """

    # Create plot
    fig = go.Figure()

    # Full equity curve
    fig.add_trace(go.Scatter(
        x=portfolio_returns['cumulative_returns'].index, y=portfolio_returns['cumulative_returns'].values,
        mode='lines', name='Full Equity Curve',
        line=dict(color='yellow', width=2)
    ))

    # Add stress overlays
    for label, (start, end) in stress_windows.items():
        stress_curve = portfolio_returns['cumulative_returns'].loc[start:end]

        fig.add_trace(go.Scatter(
            x=stress_curve.index,
            y=stress_curve.values,
            mode='lines',
            name=label,
            line=dict(dash='dashdot'),
        ))

    # Layout
    fig.update_layout(
        title="Equity Curve with Stress Period Overlays",
        xaxis_title="Date",
        yaxis_title="Portfolio Value",
        legend_title="Legend",
        template="plotly_white"
    )

    fig.show()

Plot

In [ ]:
plot_equity_curve_with_stress(portfolio_returns, STRESS_EVENTS)

### 🦾 Proposed Enhancement: Risk-Parity Weights with Dynamic Rebalancing Frequency

Import of required package

In [234]:
import requests

Initialization of API Token

In [232]:
Alpha_Vantage_API_TOKEN = "GBMV74GZ3G8DGXO2" # Replace with your own Alpha Vantage API token

Function for fetching Real GDP and FED Rate data

In [344]:
def fetch_data(data_type: str, frequency: str, start_date: str = START_DATE, end_date: str = END_DATE):
    """
    Fetches data (Real GDP or FED Rates) from Alpha Vantage API for a given frequency.

    Parameters:
        data_type (str): Type of data to fetch, 'GDP' or 'FED Rates'
        frquency (str): Frequency of data, 'quarterly' or 'annual' for GDP / 'daily', 'weekly', or 'monthly' for FED rates
        start_date (str): Start date in 'YYYY-MM-DD' format (default to START_DATE)
        end_date (str): End date in 'YYYY-MM-DD' format (default to END_DATE)

    Returns:
        pd.DataFrame: DataFrame with date-/datetime-indexed data
    """

    if data_type not in ['GDP', 'FED Rates']:
        raise ValueError("Data Type must be either 'GDP' or 'FED Rates'.")
    
    if (data_type == 'GDP' and frequency not in ['quarterly', 'annual']) or (data_type == 'FED Rates' and frequency not in ['daily', 'weekly', 'monthly']):
        raise ValueError("Frequency must be either 'quarterly' or 'annual' for GDP or 'daily', 'weekly', or 'monthly' for FED rates.")

    request_function = 'REAL_GDP' if data_type == 'GDP' else 'FEDERAL_FUNDS_RATE'
    request_url = f'https://www.alphavantage.co/query?function={request_function}&interval={frequency}&apikey={Alpha_Vantage_API_TOKEN}'
    requestResponse = requests.get(request_url)

    # Check if the response is valid
    if requestResponse.status_code != 200:
        raise ValueError(f"Error fetching GDP data.")

    # Extract Requested Data
    result = pd.DataFrame(requestResponse.json()['data'])

    # Convert 'date' column to datetime
    result["date"] = pd.to_datetime(result["date"])

    # Set 'date' as index
    result.set_index("date", inplace=True)
    
    # Ensure the DataFrame is sorted by date
    result = result.sort_index()

    # Convert the value column to numeric
    result = result.astype({'value': float})

    # Rename the column accordingly
    result.columns = ['GDP'] if data_type == 'GDP' else ['FED Rate']

    return result.loc[start_date:end_date]

Function for calculating GDP signal and FED Rates signal

In [273]:
def calculate_GDP_signal(gdp: pd.DataFrame):
    """
    Estimate the trend of GDP growth for equity exposure tilt

    Parameters:
        gdp (pd.DataFrame): Date-indexed GDP data

    Returns:
        pd.DataFrame: DataFrame with date-/datetime-indexed signal (-1 for falling, 0 for flat, 1 for rising)
    """

    # Interpolate data to monthly
    gdp_monthly = gdp.resample('ME').interpolate(method='linear')  # Assumes linear progresison within quarters

    # Compute monthly change (slope)
    gdp_diff = gdp_monthly.diff()

    # Smooth the GDP change over a 15-month window
    gdp_diff_ma15 = gdp_diff.rolling(window=15).mean()

    # Convert to directional signal
    gdp_signal = gdp_diff_ma15.apply(np.sign).reindex(gdp.index).ffill()

    return gdp_signal.dropna()


In [291]:
def calculate_FED_signal(fed: pd.DataFrame):
    """
    Estimate the trend of refinancing rate for bond duration tilt

    Parameters:
        fed (pd.DataFrame): Date-indexed FED Rates data

    Returns:
        pd.DataFrame: DataFrame with date-/datetime-indexed signal (-1 for falling, 0 for flat, 1 for rising)
    """

    # Smooth the rate using a 12-month moving average
    fed_ma12 = fed.rolling(window=252, min_periods=10).mean()

    # Compute the slope (rate of change) of the smoothed Fed rate
    fed_slope = fed_ma12.diff().rolling(window=21).mean()

    # Convert slope into a directional signal: +1, 0, or -1
    fed_signal = fed_slope.apply(np.sign).reindex(fed.index).ffill()

    return fed_signal.dropna()

Function for calculating dynamic rebalancing weights

In [357]:
def calculate_daily_dynamic_reb_weights(weights: pd.DataFrame, gdp_signal: pd.DataFrame, fed_signal:pd.DataFrame, equity_tilt: float = 0.06, duration_tilt: float = 0.10):
    """
    Calculate dynamic rebalancing weights by applying macro overlay

    Parameters:
        wights (pd.DataFrame): Date-indexed risk-parity portfolip weights
        gdp_signal (pd.DataFrame): Directional signals from GDP data
        fed_signal (pd.DataFrame): Directional signals from FED Rates data
        equity_tilt (float): Amount of allocation to shift from/to equity ETF (default to 6%)
        duration_tilt (float): Amount of allocation to shift from/to bond ETF (default to 10%)

    Returns:
        pd.DataFrame: DataFrame with date-/datetime-indexed dynamically rebalanced weights
    """

    dynamic_reb_weights = weights.copy()
    for date in dynamic_reb_weights.index:
        if dynamic_reb_weights.loc[date].isna().any():
            continue

        d_signal = fed_signal['FED Rate'].loc[date] if date in fed_signal.index else 0
        e_signal = gdp_signal['GDP'].loc[date] if date in gdp_signal.index else 0

        # Apply Equity Tilt (to SPY)
        dynamic_reb_weights.loc[date, 'SPY'] += e_signal * equity_tilt


        # Apply Duration Tilt (TLT vs TIP)
        dynamic_reb_weights.loc[date, 'TLT'] += d_signal * duration_tilt
        dynamic_reb_weights.loc[date, 'TIP'] -= d_signal * duration_tilt

        # Normalize weights to sum to 1 and clip negatives
        row = dynamic_reb_weights.loc[date].clip(lower=0)
        dynamic_reb_weights.loc[date] = row / row.sum()

    return dynamic_reb_weights

In [356]:
df = calculate_FED_signal(fetch_data('FED Rates', 'daily'))
df

,FED Rate
date,
2015-01-31,1.0
2015-02-01,-1.0
2015-02-02,-1.0
2015-02-03,-1.0
2015-02-04,-1.0
...,...
2025-06-23,-1.0
2025-06-24,-1.0
2025-06-25,-1.0


In [346]:
fetch_data('GDP', 'quarterly')


,GDP
date,
2015-01-01,4562.381
2015-04-01,4696.817
2015-07-01,4741.070
2015-10-01,4799.354
2016-01-01,4641.851
2016-04-01,4768.308
2016-07-01,4819.182
2016-10-01,4912.331
2017-01-01,4698.367


In [ ]:
calculate_daily_dynamic_reb_weights(portfolio_weights, calculate_GDP_signal(fetch_data('GDP', 'quarterly')), calculate_FED_signal(fetch_data('FED Rates', 'daily')))

/tmp/ipykernel_721/3175378141.py:19: FutureWarning:

DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.



ValueError: Incompatible indexer with Series